In [3]:
import pandas as pd
from pathlib import Path

In [8]:
CV_RES_ROOT = Path("../data/multi_modal_ppi_results/cross_validation")
HPRD_PATH = CV_RES_ROOT / "hprd_run.csv"
BIOINFER_PATH = CV_RES_ROOT / "bioinfer_run.csv"

In [32]:
hprd_df = pd.read_csv(HPRD_PATH, index_col="Name")
bioinfer_df = pd.read_csv(BIOINFER_PATH, index_col="Name")

In [33]:
MODULE_CHOICES = [
    "TextModule",
    "TextAndGraphModule",
    "TextAndNumModule",
    "TextAndGraphAndNumModule",
    "GraphModule",
    "GraphAndNumModule",
    "NumModule",
]

METRICS_CHOICES = [
    ("test/f1_mean", "F1 score"),
    ("test/auroc_mean", "AUROC")
]

In [34]:
new_index = [f"{orig}{i}" for i, orig in enumerate(hprd_df.index)]
hprd_df.index = new_index

In [46]:
for module in MODULE_CHOICES:
    print(f"Module: {module}")
    for metric in METRICS_CHOICES:
        tmp_df = hprd_df.query(f"index.str.contains('{module}')").query(f"index.str.startswith('{module[0]}')")
        max_idx = tmp_df[metric[0]].idxmax()
        max_row = tmp_df.loc[max_idx]
        print(f"    Based on {metric[1]}")
        for _metric in METRICS_CHOICES:
            print(f"    {_metric[0]}:{100*max_row[{_metric[0]}][_metric[0]]:.2f}")

Module: TextModule
    Based on F1 score
    test/f1_mean:73.34
    test/auroc_mean:94.55
    Based on AUROC
    test/f1_mean:73.34
    test/auroc_mean:94.55
Module: TextAndGraphModule
    Based on F1 score
    test/f1_mean:67.79
    test/auroc_mean:92.06
    Based on AUROC
    test/f1_mean:67.79
    test/auroc_mean:92.06
Module: TextAndNumModule
    Based on F1 score
    test/f1_mean:78.12
    test/auroc_mean:93.15
    Based on AUROC
    test/f1_mean:75.24
    test/auroc_mean:96.71
Module: TextAndGraphAndNumModule
    Based on F1 score
    test/f1_mean:79.17
    test/auroc_mean:93.72
    Based on AUROC
    test/f1_mean:74.55
    test/auroc_mean:95.93
Module: GraphModule
    Based on F1 score
    test/f1_mean:4.68
    test/auroc_mean:51.36
    Based on AUROC
    test/f1_mean:4.68
    test/auroc_mean:51.36
Module: GraphAndNumModule
    Based on F1 score
    test/f1_mean:10.25
    test/auroc_mean:57.04
    Based on AUROC
    test/f1_mean:10.25
    test/auroc_mean:57.04
Module: NumModule


In [38]:
module = "TextAndGraphAndNumModule"
tmp_df = hprd_df.query(f"index.str.contains('{module}')").query(f"index.str.startswith('{module[0]}')")
max_idx = tmp_df["test/f1_mean"].idxmax()
max_row = tmp_df.loc[max_idx]
print(max_row[["test/f1_mean", "test/auroc_mean"]])

test/f1_mean       0.791679
test/auroc_mean    0.937232
Name: TextAndGraphAndNumModule_274631721313, dtype: object


In [20]:
for module in MODULE_CHOICES:
    tmp_df = hprd_df.query(f"index.str.contains('{module}')").query(f"index.str.startswith('{module[0]}')")
    display(tmp_df.head(10))
# single_val_kwdlc_df = val_kwdlc_df.query(f'index.str.contains("1stg")')

,Start Time,Duration,Run ID,Source Type,Source Name,User,Status,best_checkpoint_0fold,best_checkpoint_1fold,best_checkpoint_2fold,...,trainer_resume_from_checkpoint,trainer_weights_summary,test/auroc_mean,test/auroc_std,test/f1_mean,test/f1_std,test/prec_mean,test/prec_std,test/rec_mean,test/rec_std
Name,,,,,,,,,,,,,,,,,,,,,
TextModule_2746317213,2022-02-03 13:57:33,48ms,9fe0573919614914b0586c4c22a15b25,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/12-58-14...,/data1/NLP_PPI/result/runs/2022-02-03/12-58-14...,/data1/NLP_PPI/result/runs/2022-02-03/12-58-14...,...,None,None,0.94548,0.085758,0.733407,0.064472,0.786138,0.110331,0.725734,0.167808


,Start Time,Duration,Run ID,Source Type,Source Name,User,Status,best_checkpoint_0fold,best_checkpoint_1fold,best_checkpoint_2fold,...,trainer_resume_from_checkpoint,trainer_weights_summary,test/auroc_mean,test/auroc_std,test/f1_mean,test/f1_std,test/prec_mean,test/prec_std,test/rec_mean,test/rec_std
Name,,,,,,,,,,,,,,,,,,,,,
NumModule_2746317213,2022-02-09 10:13:40,50ms,eb1267b7610a4e8d9306bef048f76215,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-09/10-05-17...,/data1/NLP_PPI/result/runs/2022-02-09/10-05-17...,/data1/NLP_PPI/result/runs/2022-02-09/10-05-17...,...,None,None,0.489841,0.055808,0.048873,0.040125,0.052904,0.045874,0.047852,0.039584
NumModule_2746317213,2022-02-09 10:05:16,50ms,bc3ac95cacf44f4a95499aa80ac23cc2,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-09/09-58-49...,/data1/NLP_PPI/result/runs/2022-02-09/09-58-49...,/data1/NLP_PPI/result/runs/2022-02-09/09-58-49...,...,None,None,0.498768,0.084223,0.050725,0.042116,0.050362,0.048534,0.073566,0.073650
NumModule_2746317213,2022-02-09 09:58:47,49ms,f8809ddb93294f67acf585fbf94de264,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-09/09-54-08...,/data1/NLP_PPI/result/runs/2022-02-09/09-54-08...,/data1/NLP_PPI/result/runs/2022-02-09/09-54-08...,...,None,None,0.553274,0.068573,0.040196,0.050412,0.051515,0.067488,0.033566,0.041348
NumModule_2746317213,2022-02-09 09:54:06,47ms,d3563f7e6d204ebcab3a1f1272c29bb6,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-09/09-48-46...,/data1/NLP_PPI/result/runs/2022-02-09/09-48-46...,/data1/NLP_PPI/result/runs/2022-02-09/09-48-46...,...,None,None,0.532063,0.112591,0.104188,0.028776,0.066711,0.031531,0.432767,0.151279
NumModule_2746317213,2022-02-09 09:48:45,49ms,a151dd457b0141359da526940e7ac32c,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-09/09-44-11...,/data1/NLP_PPI/result/runs/2022-02-09/09-44-11...,/data1/NLP_PPI/result/runs/2022-02-09/09-44-11...,...,None,None,0.583158,0.130161,0.043333,0.064636,0.206667,0.396877,0.038182,0.046851


,Start Time,Duration,Run ID,Source Type,Source Name,User,Status,best_checkpoint_0fold,best_checkpoint_1fold,best_checkpoint_2fold,...,trainer_resume_from_checkpoint,trainer_weights_summary,test/auroc_mean,test/auroc_std,test/f1_mean,test/f1_std,test/prec_mean,test/prec_std,test/rec_mean,test/rec_std
Name,,,,,,,,,,,,,,,,,,,,,
TextAndGraphModule_2746317213,2022-02-03 14:59:18,55ms,cd9794544c164117b747005aa743e662,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/13-57-35...,/data1/NLP_PPI/result/runs/2022-02-03/13-57-35...,/data1/NLP_PPI/result/runs/2022-02-03/13-57-35...,...,None,None,0.920552,0.081366,0.677902,0.217319,0.788889,0.170873,0.62002,0.256381


,Start Time,Duration,Run ID,Source Type,Source Name,User,Status,best_checkpoint_0fold,best_checkpoint_1fold,best_checkpoint_2fold,...,trainer_resume_from_checkpoint,trainer_weights_summary,test/auroc_mean,test/auroc_std,test/f1_mean,test/f1_std,test/prec_mean,test/prec_std,test/rec_mean,test/rec_std
Name,,,,,,,,,,,,,,,,,,,,,
TextAndGraphAndNumModule_2746317213,2022-02-05 01:34:00,65ms,04bdefeac2d148cbac79efff55eb4277,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-05/00-10-48...,/data1/NLP_PPI/result/runs/2022-02-05/00-10-48...,/data1/NLP_PPI/result/runs/2022-02-05/00-10-48...,...,None,None,0.930400,0.083384,0.739996,0.091426,0.806061,0.126549,0.710350,0.142188
TextAndGraphAndNumModule_2746317213,2022-02-05 00:10:46,65ms,f05ac6b14d3845ad9d634aae621753e0,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/22-54-16...,/data1/NLP_PPI/result/runs/2022-02-04/22-54-16...,/data1/NLP_PPI/result/runs/2022-02-04/22-54-16...,...,None,None,0.772467,0.188911,0.539654,0.275437,0.650556,0.350557,0.470350,0.239797
TextAndGraphAndNumModule_2746317213,2022-02-04 22:54:14,51ms,37801ce8791e4dd9a6d81e64ed1a7eca,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/21-32-58...,/data1/NLP_PPI/result/runs/2022-02-04/21-32-58...,/data1/NLP_PPI/result/runs/2022-02-04/21-32-58...,...,None,None,0.937232,0.068883,0.791679,0.096013,0.896111,0.064731,0.724635,0.157927
TextAndGraphAndNumModule_2746317213,2022-02-04 21:32:56,61ms,d032cbc42e8d4065bbe1754b6c28aa12,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/20-17-36...,/data1/NLP_PPI/result/runs/2022-02-04/20-17-36...,/data1/NLP_PPI/result/runs/2022-02-04/20-17-36...,...,None,None,0.849756,0.180267,0.593351,0.300812,0.601648,0.330018,0.612168,0.320405
TextAndGraphAndNumModule_2746317213,2022-02-04 20:17:34,62ms,f9fd7e2e3c464d70b32505f46d3e5ffd,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/18-57-19...,/data1/NLP_PPI/result/runs/2022-02-04/18-57-19...,/data1/NLP_PPI/result/runs/2022-02-04/18-57-19...,...,None,None,0.953531,0.043195,0.732455,0.102627,0.822857,0.121345,0.685475,0.160936
TextAndGraphAndNumModule_2746317213,2022-02-04 18:57:17,59ms,4e284cf2b42a4fa58c50d962ffcd3e58,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/17-37-18...,/data1/NLP_PPI/result/runs/2022-02-04/17-37-18...,/data1/NLP_PPI/result/runs/2022-02-04/17-37-18...,...,None,None,0.936874,0.068527,0.688785,0.060130,0.738889,0.077534,0.673267,0.130284
TextAndGraphAndNumModule_2746317213,2022-02-04 17:37:17,61ms,45376348691e48299cb04eebc2d7cbc9,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/16-16-41...,/data1/NLP_PPI/result/runs/2022-02-04/16-16-41...,/data1/NLP_PPI/result/runs/2022-02-04/16-16-41...,...,None,None,0.959273,0.048651,0.745515,0.083419,0.796667,0.172691,0.740020,0.150088
TextAndGraphAndNumModule_2746317213,2022-02-04 16:16:39,61ms,ca252bafa7424048b22f2b9d28eb2c02,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/14-56-49...,/data1/NLP_PPI/result/runs/2022-02-04/14-56-49...,/data1/NLP_PPI/result/runs/2022-02-04/14-56-49...,...,None,None,0.921453,0.076369,0.703131,0.150792,0.832463,0.109591,0.665674,0.229628
TextAndGraphAndNumModule_2746317213,2022-02-04 14:56:47,59ms,d19b29067d2346efaacacfb38cdace37,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/13-38-24...,/data1/NLP_PPI/result/runs/2022-02-04/13-38-24...,/data1/NLP_PPI/result/runs/2022-02-04/13-38-24...,...,None,None,0.943738,0.080398,0.759749,0.076261,0.820000,0.106667,0.724635,0.122571


,Start Time,Duration,Run ID,Source Type,Source Name,User,Status,best_checkpoint_0fold,best_checkpoint_1fold,best_checkpoint_2fold,...,trainer_resume_from_checkpoint,trainer_weights_summary,test/auroc_mean,test/auroc_std,test/f1_mean,test/f1_std,test/prec_mean,test/prec_std,test/rec_mean,test/rec_std
Name,,,,,,,,,,,,,,,,,,,,,
GraphModule_2746317213,2022-02-05 01:37:15,57ms,86fe29726cdc4d07874c0a25ec963a68,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-05/01-34-02...,/data1/NLP_PPI/result/runs/2022-02-05/01-34-02...,/data1/NLP_PPI/result/runs/2022-02-05/01-34-02...,...,None,None,0.513647,0.078571,0.04684,0.041396,0.025664,0.022815,0.271329,0.228336


,Start Time,Duration,Run ID,Source Type,Source Name,User,Status,best_checkpoint_0fold,best_checkpoint_1fold,best_checkpoint_2fold,...,trainer_resume_from_checkpoint,trainer_weights_summary,test/auroc_mean,test/auroc_std,test/f1_mean,test/f1_std,test/prec_mean,test/prec_std,test/rec_mean,test/rec_std
Name,,,,,,,,,,,,,,,,,,,,,
GraphAndNumModule_2746317213,2022-02-05 02:18:21,58ms,73a2099c61ec4516b4e611996a46322d,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-05/02-07-36...,/data1/NLP_PPI/result/runs/2022-02-05/02-07-36...,/data1/NLP_PPI/result/runs/2022-02-05/02-07-36...,...,None,None,0.456127,0.068728,0.074717,0.030872,0.041956,0.017627,0.346274,0.127215
GraphAndNumModule_2746317213,2022-02-05 02:07:35,54ms,8eaec50b286a4957a1a254879c8939ed,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-05/01-58-48...,/data1/NLP_PPI/result/runs/2022-02-05/01-58-48...,/data1/NLP_PPI/result/runs/2022-02-05/01-58-48...,...,None,None,0.570404,0.055631,0.102462,0.019823,0.062621,0.012254,0.494306,0.212908
GraphAndNumModule_2746317213,2022-02-05 01:58:46,58ms,b71b372db8e645dbb1b4a39e7182794e,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-05/01-52-00...,/data1/NLP_PPI/result/runs/2022-02-05/01-52-00...,/data1/NLP_PPI/result/runs/2022-02-05/01-52-00...,...,None,None,0.480934,0.069608,0.070613,0.047152,0.040025,0.026568,0.304316,0.213160
GraphAndNumModule_2746317213,2022-02-05 01:51:58,57ms,01e8ddcbad814a86aae033b50a63eab9,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-05/01-44-21...,/data1/NLP_PPI/result/runs/2022-02-05/01-44-21...,/data1/NLP_PPI/result/runs/2022-02-05/01-44-21...,...,None,None,0.480655,0.113553,0.080932,0.026083,0.044415,0.015522,0.494506,0.086837
GraphAndNumModule_2746317213,2022-02-05 01:44:19,56ms,b27c2de26c7449979d6db4ad18c9ff3d,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-05/01-37-16...,/data1/NLP_PPI/result/runs/2022-02-05/01-37-16...,/data1/NLP_PPI/result/runs/2022-02-05/01-37-16...,...,None,None,0.427829,0.039897,0.070538,0.032618,0.039980,0.018680,0.346873,0.194086


,Start Time,Duration,Run ID,Source Type,Source Name,User,Status,best_checkpoint_0fold,best_checkpoint_1fold,best_checkpoint_2fold,...,trainer_resume_from_checkpoint,trainer_weights_summary,test/auroc_mean,test/auroc_std,test/f1_mean,test/f1_std,test/prec_mean,test/prec_std,test/rec_mean,test/rec_std
Name,,,,,,,,,,,,,,,,,,,,,
TextAndNumModule_2746317213,2022-02-04 02:30:08,54ms,d7fd78c6e1674bbcbbb379457b419f7a,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/01-22-09...,/data1/NLP_PPI/result/runs/2022-02-04/01-22-09...,/data1/NLP_PPI/result/runs/2022-02-04/01-22-09...,...,None,None,0.942678,0.068004,0.759838,0.105906,0.829870,0.146839,0.724635,0.157927
TextAndNumModule_2746317213,2022-02-04 01:22:07,50ms,a7beba4998644f398296708cbb922c0c,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-04/00-13-11...,/data1/NLP_PPI/result/runs/2022-02-04/00-13-11...,/data1/NLP_PPI/result/runs/2022-02-04/00-13-11...,...,None,None,0.926148,0.063267,0.719814,0.090071,0.750583,0.133507,0.721838,0.166186
TextAndNumModule_2746317213,2022-02-04 00:13:09,54ms,8d8e70fc0caf45af944810ebd247caaa,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/23-02-40...,/data1/NLP_PPI/result/runs/2022-02-03/23-02-40...,/data1/NLP_PPI/result/runs/2022-02-03/23-02-40...,...,None,None,0.931481,0.075556,0.781214,0.065079,0.897222,0.055277,0.706454,0.127476
TextAndNumModule_2746317213,2022-02-03 23:02:39,55ms,4740c69b4622487bbc9b3d554ab00bc7,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/21-51-16...,/data1/NLP_PPI/result/runs/2022-02-03/21-51-16...,/data1/NLP_PPI/result/runs/2022-02-03/21-51-16...,...,None,None,0.955108,0.065862,0.758615,0.065933,0.778106,0.144785,0.766773,0.117229
TextAndNumModule_2746317213,2022-02-03 21:51:14,55ms,e31f0888226e40b986f4070fbd367618,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/20-46-58...,/data1/NLP_PPI/result/runs/2022-02-03/20-46-58...,/data1/NLP_PPI/result/runs/2022-02-03/20-46-58...,...,None,None,0.852929,0.197961,0.603334,0.322089,0.678788,0.354384,0.569091,0.337105
TextAndNumModule_2746317213,2022-02-03 20:46:57,54ms,ca01ac65259c497ca144159acaff79b1,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/19-41-26...,/data1/NLP_PPI/result/runs/2022-02-03/19-41-26...,/data1/NLP_PPI/result/runs/2022-02-03/19-41-26...,...,None,None,0.830858,0.173730,0.615654,0.317714,0.709921,0.361430,0.568531,0.329085
TextAndNumModule_2746317213,2022-02-03 19:41:25,56ms,fa1c972c41ca43439e7577a2c54e650d,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/18-29-36...,/data1/NLP_PPI/result/runs/2022-02-03/18-29-36...,/data1/NLP_PPI/result/runs/2022-02-03/18-29-36...,...,None,None,0.967123,0.044354,0.752375,0.089897,0.857374,0.105769,0.687552,0.131949
TextAndNumModule_2746317213,2022-02-03 18:29:35,55ms,538b66db49584eee93d753b262719aa5,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/17-17-38...,/data1/NLP_PPI/result/runs/2022-02-03/17-17-38...,/data1/NLP_PPI/result/runs/2022-02-03/17-17-38...,...,None,None,0.916147,0.079762,0.760958,0.107337,0.822980,0.067393,0.725734,0.167808
TextAndNumModule_2746317213,2022-02-03 17:17:36,55ms,ecdc7229a3c34cff9bca075b7654ccc5,LOCAL,/home/umakoshi/Documents/ppi/gcn_ppi/run.py,umakoshi,FINISHED,/data1/NLP_PPI/result/runs/2022-02-03/16-06-43...,/data1/NLP_PPI/result/runs/2022-02-03/16-06-43...,/data1/NLP_PPI/result/runs/2022-02-03/16-06-43...,...,None,None,0.964443,0.049794,0.746296,0.056704,0.816692,0.102066,0.725734,0.167808
